# GNOD Part_5

### Pseudocode

0. Import:
    - pickle scaler
    - pickle kmeans
    - database with top_100_songs
    - database with all songs with cluster
1. User_input:
    - hot - another hot song
    - not hot - next step
2. Not hot:
    - Query with the song title to get a song features and audio fromt the spotify
3. Song from a query:
    - preparing the same way like our playlist
    - using the scaler
    - using kmeans to cluster the song
4. Checking to which cluster the songs belong to.
5. Proposing a random song from the same cluster.


In [5]:
import pandas as pd 
from pandas import json_normalize
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from sklearn import cluster, datasets
from sklearn.preprocessing import StandardScaler

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score 

import pickle 
from pickle import dump
from pickle import load

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from random import randint
from time import sleep

from bs4 import BeautifulSoup
import requests

import re

### 0. Importing tools

In [6]:
# Opening the pickle with scaler
scaler = load(open('scaler.p','rb'))     # reading a file 

In [8]:
# Opening the pickle with kmeans
kmeans_cluster = load(open('kmeans_cluster_songs.p','rb')) 

In [10]:
df = pd.read_csv("df_final.csv")
df

,song_name,name,song_id,popularity,danceability,energy,key,loudness,mode,speechiness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,cluster
0,2K,Nosaj Thing,33xMbeHzmWd6Od0BmLZEUs,0,0.310,0.445,7,-13.355,0,0.0863,...,0.122,95.360,audio_features,33xMbeHzmWd6Od0BmLZEUs,spotify:track:33xMbeHzmWd6Od0BmLZEUs,https://api.spotify.com/v1/tracks/33xMbeHzmWd6...,https://api.spotify.com/v1/audio-analysis/33xM...,152560,3,7
1,4 Billion Souls,The Doors,3UnyplmZaq547hwsfOR5yy,26,0.419,0.565,5,-11.565,1,0.0347,...,0.648,151.277,audio_features,3UnyplmZaq547hwsfOR5yy,spotify:track:3UnyplmZaq547hwsfOR5yy,https://api.spotify.com/v1/tracks/3UnyplmZaq54...,https://api.spotify.com/v1/audio-analysis/3Uny...,197707,4,1
2,4 Minute Warning,Radiohead,1w8QCSDH4QobcQeT4uMKLm,0,0.354,0.302,9,-13.078,1,0.0326,...,0.223,123.753,audio_features,1w8QCSDH4QobcQeT4uMKLm,spotify:track:1w8QCSDH4QobcQeT4uMKLm,https://api.spotify.com/v1/tracks/1w8QCSDH4Qob...,https://api.spotify.com/v1/audio-analysis/1w8Q...,244285,4,2
3,7 Element,Vitas,7J9mBHG4J2eIfDAv5BehKA,0,0.727,0.785,5,-6.707,0,0.0603,...,0.960,129.649,audio_features,7J9mBHG4J2eIfDAv5BehKA,spotify:track:7J9mBHG4J2eIfDAv5BehKA,https://api.spotify.com/v1/tracks/7J9mBHG4J2eI...,https://api.spotify.com/v1/audio-analysis/7J9m...,249940,4,7
4,#9 Dream,R.E.M.,1VZedwJj1gyi88WFRhfThb,6,0.571,0.724,0,-5.967,1,0.0260,...,0.385,116.755,audio_features,1VZedwJj1gyi88WFRhfThb,spotify:track:1VZedwJj1gyi88WFRhfThb,https://api.spotify.com/v1/tracks/1VZedwJj1gyi...,https://api.spotify.com/v1/audio-analysis/1VZe...,278320,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16463,Apologize,OneRepublic,6ucR4KfvsBFWCMVFDvyKKl,75,0.653,0.604,8,-6.017,1,0.0278,...,0.101,118.016,audio_features,6ucR4KfvsBFWCMVFDvyKKl,spotify:track:6ucR4KfvsBFWCMVFDvyKKl,https://api.spotify.com/v1/tracks/6ucR4KfvsBFW...,https://api.spotify.com/v1/audio-analysis/6ucR...,184400,4,1
16464,Goddamn Lonely Love - Remastered,Drive-By Truckers,241LK9hqfXJyWpQ7oClQSN,48,0.427,0.541,7,-7.042,1,0.0294,...,0.153,119.094,audio_features,241LK9hqfXJyWpQ7oClQSN,spotify:track:241LK9hqfXJyWpQ7oClQSN,https://api.spotify.com/v1/tracks/241LK9hqfXJy...,https://api.spotify.com/v1/audio-analysis/241L...,301720,4,0
16465,Graveless yet Dead,Convocation,6hF9etLMoADWLsoui5ejGW,19,0.262,0.689,2,-8.342,1,0.0705,...,0.178,130.110,audio_features,6hF9etLMoADWLsoui5ejGW,spotify:track:6hF9etLMoADWLsoui5ejGW,https://api.spotify.com/v1/tracks/6hF9etLMoADW...,https://api.spotify.com/v1/audio-analysis/6hF9...,572149,4,8
16466,Gimme! Gimme! Gimme! (A Man After Midnight),ABBA,3vkQ5DAB1qQMYO4Mr9zJN6,84,0.749,0.491,10,-9.655,1,0.0403,...,0.536,119.528,audio_features,3vkQ5DAB1qQMYO4Mr9zJN6,spotify:track:3vkQ5DAB1qQMYO4Mr9zJN6,https://api.spotify.com/v1/tracks/3vkQ5DAB1qQM...,https://api.spotify.com/v1/audio-analysis/3vkQ...,292613,4,1


In [11]:
df.columns

Index(['song_name', 'name', 'song_id', 'popularity', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri',
       'track_href', 'analysis_url', 'duration_ms', 'time_signature',
       'cluster'],
      dtype='object')

In [19]:
hot_songs = pd.read_csv("top_100_songs.csv")
hot_songs

,title,artist
0,texas hold 'em,beyoncé
1,lose control,teddy swims
2,beautiful things,benson boone
3,texas hold 'em,beyoncé
4,dont let me go,mgk
...,...,...
95,can't get enough,jennifer lopez
96,unstoppable,sia
97,until i found you (em beihold version),stephen sanchez & em beihold
98,greedy,tate mcrae


### Rearanging the dataframe with database
- I keep only some features: song_name, name, cluster, url


In [16]:
df2 = df[["song_name", "name", "uri", "cluster"]]
df2

,song_name,name,uri,cluster
0,2K,Nosaj Thing,spotify:track:33xMbeHzmWd6Od0BmLZEUs,7
1,4 Billion Souls,The Doors,spotify:track:3UnyplmZaq547hwsfOR5yy,1
2,4 Minute Warning,Radiohead,spotify:track:1w8QCSDH4QobcQeT4uMKLm,2
3,7 Element,Vitas,spotify:track:7J9mBHG4J2eIfDAv5BehKA,7
4,#9 Dream,R.E.M.,spotify:track:1VZedwJj1gyi88WFRhfThb,1
...,...,...,...,...
16463,Apologize,OneRepublic,spotify:track:6ucR4KfvsBFWCMVFDvyKKl,1
16464,Goddamn Lonely Love - Remastered,Drive-By Truckers,spotify:track:241LK9hqfXJyWpQ7oClQSN,0
16465,Graveless yet Dead,Convocation,spotify:track:6hF9etLMoADWLsoui5ejGW,8
16466,Gimme! Gimme! Gimme! (A Man After Midnight),ABBA,spotify:track:3vkQ5DAB1qQMYO4Mr9zJN6,1


In [17]:
# Renaming columns
df2 = df2.rename(columns={'song_name':'title',
                            'name':'artist', 
                            'uri':'uri',
                         'cluster' : 'cluster'})

In [18]:
df2

,title,artist,uri,cluster
0,2K,Nosaj Thing,spotify:track:33xMbeHzmWd6Od0BmLZEUs,7
1,4 Billion Souls,The Doors,spotify:track:3UnyplmZaq547hwsfOR5yy,1
2,4 Minute Warning,Radiohead,spotify:track:1w8QCSDH4QobcQeT4uMKLm,2
3,7 Element,Vitas,spotify:track:7J9mBHG4J2eIfDAv5BehKA,7
4,#9 Dream,R.E.M.,spotify:track:1VZedwJj1gyi88WFRhfThb,1
...,...,...,...,...
16463,Apologize,OneRepublic,spotify:track:6ucR4KfvsBFWCMVFDvyKKl,1
16464,Goddamn Lonely Love - Remastered,Drive-By Truckers,spotify:track:241LK9hqfXJyWpQ7oClQSN,0
16465,Graveless yet Dead,Convocation,spotify:track:6hF9etLMoADWLsoui5ejGW,8
16466,Gimme! Gimme! Gimme! (A Man After Midnight),ABBA,spotify:track:3vkQ5DAB1qQMYO4Mr9zJN6,1


### 1. User input

In [24]:
def clean_user_input(user_input):
    # Function to clean whitestrips and make values to lowercase
    user_input = user_input.lower()
    user_input = user_input.strip()
    return user_input

In [25]:
user_input = input("Please give a name of your favourite song: ")
user_input = clean_user_input(user_input)
user_input

Please give a name of your favourite song: LOSE


'lose'

In [27]:
user_input = input("Please give a name of your favourite song: ")

Please give a name of your favourite song: LOSE


In [29]:
def user_input_check(user_input):
    user_input = input("Please give a name of your favourite song: ")
    user_input = clean_user_input(user_input)

    while len(user_input) <= 3:
        print("Too short, put again")
        user_input = input("Please give a name of your favourite song: ")
    if len(user_input)>3:
        user_input = clean_user_input(user_input)
        print(user_input)

In [39]:
def get_input():
    user_input = input("Please give a name of your favourite song: ")
    return user_input

def check_length(input_string):
    ergebnis = True
    if len(input_string) <= 3:
        ergebnis = False
    else:
        ergebnis = True
        
    return ergebnis

def check_fav_song():
    song_name = get_input()
    is_name_valid = check_length(song_name)
    
    if is_name_valid:
        print("all good")
    else:
        print("too short")
        
check_fav_song()

Please give a name of your favourite song: hgfd
all good


## 2. Not HOT..... query of a new song

In [21]:
secrets_file = open("secrets.txt","r")
string = secrets_file.read()

secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
#         print(line.split(':'))
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()
    

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

In [22]:
# results = sp.search(q=user_input, limit=1)

In [30]:
# results